In [23]:
"""You can define global variables here """

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib

global model # We want to use the same model throughout the interface
global company_num

def start( company, name ):
    """Called once at the beginning of testing period.
        Company is the number of the company you are predicting (0-2), or 3 if you are to
        predict the value of all companies.
        Name is the (path and) name of your model.
        You should load the model you use throughout here (no training to take place) """
    print("Starting model of company %d" % company)
    file_path = "models/" + name
    print("Loading model from \"" + file_path + "\"")
    from keras.models import load_model
    global model 
    global company_num
    company_num = company
    model = load_model(file_path)
    return


def predict( stock_prices, market_analysis, market_segments, info ):
    """Called on consecutive days of the test period (when stock markets are open).
       The arguments are provided as lists of tuples, ordered in an increasing
       order by date (i.e., the last tuple in stock_prices and info, are for the day you are
       predicting. You are not allowed to change the arguments.

       You are to predict the stock-price movement for the date indicated by the last record in
       the info list. Note, that the lists also contain information about older dates, but you are
       free to ignore them if not using them.

       Returns a 3-list of predictions (company 0, 1, 2, respectively). Each value is True if
       prediction is that stock price for respective company will go up, otherwise False
       (i.e. if you think company 0 will go up, but not 1 and 2, then return [True, False, False]).
       If you are only predicting a single company, set the respective field but keep the others False
       (i.e. if company is number 1 and you predict its stock will go up, then return [False, True, False]).
    """
    prediction = [False, False, False]
    # Getting the info of today
    info_today = info[len(info) - 1]
    
    # Getting stock_price of today
    stock_today = stock_prices[len(stock_prices) - 1]
    
    # If it is a single company we run this...
    if company_num != 3:
        # Get the info we use in our model
        X = info_today[4:]
        
        if company_num != 1:
            # IT = [1,0]
            X = np.hstack((X, [1, 0]))
        else:
            X = np.hstack((X, [0, 1]))
        
        # Make the stock price binary based on yesterday
        if stock_today[4] > stock_prices[len(stock_prices) - 2][4]:
            X = np.hstack((X, [1]))
        else:
            X = np.hstack((X, [0]))
        
        scaler = joblib.load("models/scaler Company 0.save")
        X = X.reshape(1, -1)
        X = scaler.transform(X)
        X_test = np.array(X[0][:-1])
        
        predict = model.predict_classes(X_test.reshape(1,-1))
        if predict[0][0] == 1:
            prediction[company_num] = True
    return prediction

def end():
    """Called once at the end of the testing period. Optional if you do anything here."""
    ...
    return


def main():
    start(0, "c1.h5")
    
    # Testing the last three tuples
    # * Stock prices apart and together *
    stock_prices_0 = [(0, 2019, 177, 1, 164.0),
                      (0, 2019, 178, 1, 164.0),
                      (0, 2019, 179, 1, 166.5)]
    stock_prices_1 = [(1, 2019, 177, 1, 134.4),
                      (1, 2019, 178, 1, 132.4),
                      (1, 2019, 179, 1, 130.8)]
    stock_prices_2 = [(2, 2019, 177, 1,  10.4),
                      (2, 2019, 178, 1,  10.7),
                      (2, 2019, 179, 1,  11.5)]
    
    stock_prices = np.hstack((stock_prices_0, stock_prices_1, stock_prices_2))
    np.set_printoptions(suppress=True)
    #print(stock_prices)
    
    # * Info apart and together *
    info_0 = [(0, 2019, 177, 1, 0, 0,  4,   8.1,  1631,  0.0, 0),
              (0, 2019, 178, 1, 0, 0,  5,   4.3,   352, -0.9, 0),
              (0, 2019, 179, 1, 0, 1,  5,  -1.2,  5765,  0.1, 0)]
    
    info_1 = [(1, 2019, 177, 1, 0, 1,  4,  -2.9,  5274,  0.5, 0),
              (1, 2019, 178, 1, 0, 0,  4,   6.8,  3373, -0.8, 0),
              (1, 2019, 179, 1, 0, 0,  4,  -0.2,  3078,  0.5, 0)]
    
    info_2 = [(0, 2019, 177, 1, 0, 0,  4,   8.1,  1631,  0.0, 0),
              (0, 2019, 178, 1, 0, 0,  5,   4.3,   352, -0.9, 0),
              (0, 2019, 179, 1, 0, 1,  5,  -1.2,  5765,  0.1, 0)]
    
    info = np.hstack((info_0, info_1, info_2))
    #print(info)
    
    print(predict( stock_prices_0, 0, 0, info_0))
    

In [24]:
main()

Starting model of company 0
Loading model from "models/c1.h5"
[False, False, False]
